In [3]:
#from __future__ import unicode_literals
import twitter_credentials as cred
import tweepy
import pickle
import os
import datetime
import time
import re
import boto

altered code from https://github.com/janvanzeghbroeck/urban-emoji/blob/master/twitter_api.py

In [4]:
auth = tweepy.OAuthHandler(cred.consumer_key, cred.consumer_secret)
auth.set_access_token(cred.access_token, cred.access_token_secret)

api = tweepy.API(auth)



def get_tweets(topic, save_file_name, num_batches=25, num_tweets = 20, to_bucket = False): # num_batches * 100 is total tweets target
    tweets = set()
    # public_tweets = api.home_timeline()
    for i in range(num_batches):
        try:
            print('Loading', i+1, 'of', num_batches)
            for tweet in tweepy.Cursor(api.search, q=topic).items(num_tweets): #100 batches of 20

                if tweet.lang == 'en':
                    tweets.add(tweet.text) 

            time.sleep(35) 
        except:
            print('Waiting for API to allow more calls...')
            time.sleep(60)
            pass

    # if to_bucket:
        pass
    else:
        pickle.dump( tweets, open( "{}.pkl".format(save_file_name), "wb" ) )
        print('Succesfully pickled', len(tweets), 'tweets!')



if __name__ == '__main__':
    now = datetime.datetime.today().ctime()
    now = re.sub(' ','_',now)
    now = re.sub(':','-',now)


    # use boto to connect to aws buckets
    conn = boto.connect_s3(cred.aws_access_key, cred.aws_access_secret_key)

    # what bucket?
    bucket_name = 'emoji-tweets'

    # check if bucket exists if not make it
    if conn.lookup(bucket_name) is None:
        b = conn.create_bucket(bucket_name)
    else:
        b = conn.get_bucket(bucket_name)

    simple_words = ['is', 'it', 'the', 'are','climate', 'Greta', 'Ocasio',
                   '🤯', '🙊', '🇰🇷', '🇺🇸', '🇩🇪']
    
    for word in simple_words:
        pkl_name = './tweet_data/tweets_{}_{}'.format(now,word)
        s3_name = 'tweets_{}_{}.pkl'.format(now,word)
        loc_name = './tweet_data/tweets_{}_{}.pkl'.format(now,word)
        get_tweets(word, pkl_name, num_batches = 10, num_tweets = 50)

        # save the pkl file
        file_object = b.new_key(s3_name)#where to save
        file_object.set_contents_from_filename(loc_name)

        print('Successfully saved {} to S3 bucket {}'.format(s3_name,bucket_name))

    
    # to read the file
    #fil_object.get_contents_to_file('folder/file')

Loading 1 of 10
Loading 2 of 10
Loading 3 of 10
Loading 4 of 10
Loading 5 of 10
Loading 6 of 10
Loading 7 of 10
Loading 8 of 10
Loading 9 of 10
Loading 10 of 10
Succesfully pickled 457 tweets!
Successfully saved tweets_Sat_Jun__8_14-13-36_2019_is.pkl to S3 bucket emoji-tweets
Loading 1 of 10
Loading 2 of 10
Loading 3 of 10
Loading 4 of 10
Loading 5 of 10
Loading 6 of 10
Loading 7 of 10
Loading 8 of 10
Loading 9 of 10
Loading 10 of 10
Succesfully pickled 443 tweets!
Successfully saved tweets_Sat_Jun__8_14-13-36_2019_it.pkl to S3 bucket emoji-tweets
Loading 1 of 10
Loading 2 of 10
Loading 3 of 10
Loading 4 of 10
Loading 5 of 10
Loading 6 of 10
Loading 7 of 10
Loading 8 of 10
Loading 9 of 10
Loading 10 of 10
Succesfully pickled 461 tweets!
Successfully saved tweets_Sat_Jun__8_14-13-36_2019_the.pkl to S3 bucket emoji-tweets
Loading 1 of 10
Loading 2 of 10
Loading 3 of 10
Loading 4 of 10
Loading 5 of 10
Loading 6 of 10
Loading 7 of 10
Loading 8 of 10
Loading 9 of 10
Loading 10 of 10
Succesf

In [1]:
from os import listdir 
from os.path import isfile, join 
import pandas as pd
mypath = './tweet_data'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [4]:
tweets = []
for i in files:
    file = './tweet_data/'+ i
    tweets += list(pickle.load(open(file,'rb')))

In [5]:
# read this many tweets!
len(tweets)

17935

In [31]:
# ----- creates the emoji df and adds the unichar
def df_emojis():
    # create full df_emoji
    df = pd.read_pickle('./data/emoji.pkl')

    df = pd.DataFrame(df[1:])
    df.rename(columns={0: 'unichar'}, inplace = True)
    
    return df

# ------------ finding the tweets with emojis
def yay_no(tweets,df_emojis):
    no_moji = []
    yay_moji = []
    for tweet in tweets:
        tweet = str(tweet) #some have type tweepy.models.Status
        yay = False
        for uni in df_emojis['unichar']:
            #if emoji in str(tweet):
            if uni in tweet:
                yay = True
        if yay:
            yay_moji.append(tweet)
        else: # else statement to create no_moji list
            no_moji.append(tweet)
    return yay_moji, no_moji


if __name__ == '__main__':

    tweets = list(set(tweets))

    df_emojis = df_emojis()
    df_emojis = df_emojis.iloc[0:1013,:] #dont emojis after 1013 yet

    yay_moji, no_moji = yay_no(tweets,df_emojis)

    pickle.dump( yay_moji, open( "./data/yay_moji.pkl", "wb"))
    pickle.dump( df_emojis, open( "./data/df_emojis.pkl", "wb"))

    print('Succesfully pickled {} tweets and emoji data frame'.format(len(yay_moji)))


Succesfully pickled 2342 tweets and emoji data frame
